In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.6MB 41.5MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-06-28 13:53:14--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz


In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from torchtext.data import TabularDataset
from torchtext.data import Iterator

In [35]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f9215b21048>)

In [36]:

tokenizer = Mecab() #토크나이져로 Mecab 활용

# Text 형태의 영화 리뷰들과 그에 해당하는 Label을 텐서로 바꿔줄 때 필요한 설정.
# torch text를 사용해서 이러한 설정정보를 담고 있는 ID, TEXT, LABEL 객체 설정 (총3개의 필드를 정의한다) 
ID = data.Field(sequential = False,
                use_vocab = False) #ID는 실제로 사용되지 않는다.

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs,
                  lower=True,
                  batch_first=True,
                  fix_length=20)
                  #include_lengths= True) #for packed pad

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)




In [37]:
# Make Dataset + Tokenizing
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

#print('train : {}'.format(len(train_data)))
#print('test : {}'.format(len(test_data)))
print(len(train_data))

150000


In [16]:
# Make Vocab.
TEXT.build_vocab(train_data, min_freq=10, vectors="glove.6B.100d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [38]:
# use above glove or here just word2vec
# Make Vocab.
TEXT.build_vocab(train_data, min_freq=5)
LABEL.build_vocab(train_data)

In [39]:
# Make validation data (80% training data, 20% validation data)
train_data, val_data = train_data.split(split_ratio=0.8)
#train_data, val_data = train_data.split(split_ratio=0.8, random_state = random.seed(SEED))


# Make Iterator (for Batch)
#train_iter = Iterator(dataset=train_data, batch_size = BATCH_SIZE, sort_within_batch = True, sort_key = lambda x: len(x.text), shuffle=True)
#test_iter = Iterator(dataset=test_data, batch_size = BATCH_SIZE, sort_within_batch = True, sort_key = lambda x: len(x.text),shuffle=False)
#val_iter = Iterator(dataset=val_data, batch_size = BATCH_SIZE, sort_within_batch = True, sort_key = lambda x: len(x.text), shuffle=False)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x : len(x.text),
    device=DEVICE)
#train_iter = Iterator(dataset=train_data, batch_size = BATCH_SIZE)
#test_iter = Iterator(dataset=test_data, batch_size = BATCH_SIZE)
#val_iter = Iterator(dataset=val_data, batch_size = BATCH_SIZE)

# Define number of words and number of lables in the 'word vocabuary'
vocab_size = len(TEXT.vocab)
n_classes = 2

print("[train]: %d [val]: %d [test]: %d [words]: %d [class] %d"
      % (len(train_data),len(val_data), len(test_data), vocab_size, n_classes))


[train]: 120000 [val]: 30000 [test]: 50000 [words]: 16037 [class] 2


In [ ]:
a,b = next(iter(train_iter)).text
print(a)
print(b)
print(len(torch.nonzero(b)))

tensor([[ 934,    5,   23,  ..., 2668,  132,    9],
        [4112,   10, 7875,  ...,    0,  886, 4243],
        [2381, 3612,   62,  ...,   79,  543,  170],
        ...,
        [2360,  822,  332,  ...,    8,    4,   18],
        [5518, 7153,    3,  ..., 2270,  182,   11],
        [5045,    3,  146,  ..., 1615,   14,  715]], device='cuda:0')
tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20], device='cuda:0')
64


In [ ]:
'''
# model = RNN(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(RNN, self).__init__()
        self.n_layers = n_layers # number of layer
        self.hidden_dim = hidden_dim # hidden layer의 차원값

        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)
        self.embed = nn.Embedding(n_vocab, embed_dim) # n_vocab = number of words in the vocab. / embeded_dim = 임베딩된 단어 텐서가 가지는 차원(임베딩할 벡터의 차원)
        self.out = nn.Linear(self.hidden_dim, n_classes) # 압축된 텐서를 신경망에 통과시켜 클래스에 대한 예측 출력

    def forward(self, x):
        # |x| = (batch_size, max_length) = ( 64, 20)
        
        embedded_x = self.embed(x)
        # |embedded_x| = (batch_size, max_length, embed_dim)
        h_0 = self._init_state(batch_size=x.size(0)) # Define 1st hidden layer
        # |h_0| , |hidden| = (1, batch_size, hid_dim) = (1, 64, 256)
        output, hidden = self.rnn(embedded_x)  # [i, b, h]
        # |output| = (batch_size, max_length, hidden_dim) = (64, 20, 256)
        # output is the concatenation of the hidden state from every step, hidden is the final hidden state
        # output[:, -1, :] == hidden.squeeze(0) = (64, 256)
        out = self.dropout(hidden.squeeze(0))
        logit = self.out(out)  # [b, h] -> [b, out_dim] (64, 2)
        #print(logit.size())
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
  '''

In [40]:

# For LSTM
# model = RNN(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2, bidirectional = False):
        super(RNN, self).__init__()
        self.n_layers = n_layers # number of layer
        self.hidden_dim = hidden_dim # hidden layer의 차원값

        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.LSTM(embed_dim, self.hidden_dim, num_layers=self.n_layers, bidirectional = bidirectional,batch_first=True)
        self.embed = nn.Embedding(n_vocab, embed_dim) # n_vocab = number of words in the vocab. / embeded_dim = 임베딩된 단어 텐서가 가지는 차원(임베딩할 벡터의 차원)
        self.out = nn.Linear(self.hidden_dim*2, n_classes) # 압축된 텐서를 신경망에 통과시켜 클래스에 대한 예측 출력

    def forward(self, x):
        # |x| = (batch_size, max_length) = ( 64, 20)
        
        embedded_x = self.embed(x)
        # |embedded_x| = (batch_size, max_length, embed_dim)
        h_0 = self._init_state(batch_size=x.size(0)) # Define 1st hidden layer
        # |h_0| , |hidden| = (1, batch_size, hid_dim) = (1, 64, 256)
        output, (hidden, cell) = self.rnn(embedded_x)  # [i, b, h]
        # |output| = (batch_size, max_length, hidden_dim) = (64, 20, 256)
        # output is the concatenation of the hidden state from every step, hidden is the final hidden state
        # output[:, -1, :] == hidden.squeeze(0) = (64, 256)
        #out = self.dropout(hidden.squeeze(0))
        out = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        logit = self.out(out)  # [b, h] -> [b, out_dim] (64, 2)
        #print(logit.size())
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
  

In [10]:
'''
# For LSTM packed padded sequence
# model = RNN(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes,  pad_idx, dropout_p=0.2, bidirectional = False):
        super(RNN, self).__init__()
        self.n_layers = n_layers # number of layer
        self.hidden_dim = hidden_dim # hidden layer의 차원값

        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.LSTM(embed_dim, self.hidden_dim, num_layers=self.n_layers, bidirectional = bidirectional)
        self.embed = nn.Embedding(n_vocab, embed_dim, padding_idx= pad_idx) # n_vocab = number of words in the vocab. / embeded_dim = 임베딩된 단어 텐서가 가지는 차원(임베딩할 벡터의 차원)
        self.out = nn.Linear(self.hidden_dim*2, n_classes) # 압축된 텐서를 신경망에 통과시켜 클래스에 대한 예측 출력

    def forward(self, x, text_lengths):
        # |x| = (batch_size, max_length) = ( 64, 20)
        
        embedded_x = self.embed(x)
        embedded_x = embedded_x.view( embedded_x.size(1), embedded_x.size(0), -1)
        #print(embedded_x.size())
        #print(text_lengths)
        #print(text_lengths.size())
        # |embedded_x| = (batch_size, max_length, embed_dim)
        #print(x)
        #print(text_lengths)
        packed_emb = nn.utils.rnn.pack_padded_sequence(embedded_x, text_lengths)
        packed_out, (hidden, cell) = self.rnn(packed_emb)
        output, output_len = nn.utils.rnn.pad_packed_sequence(packed_out)

        
        # |output| = (batch_size, max_length, hidden_dim*num directions) = (64, 20, 256*?)
        # output is the concatenation of the hidden state from every step, hidden is the final hidden state
        # output[:, -1, :] == hidden.squeeze(0) = (64, 256)
        #out = self.dropout(hidden.squeeze(0))
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #print(hidden.size())
        logit = self.out(hidden)  # [b, h] -> [b, out_dim] (64, 2)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
  '''

In [41]:
## packed pad train
'''
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        #x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text = text.to(DEVICE)
        label = batch.label.to(DEVICE)
        if(len(torch.nonzero(text_lengths))==64) : ##there are exception that length = 0 ;
          logit = model(text, text_lengths).squeeze(1)
          loss = F.cross_entropy(logit, label)
          loss.backward()
          optimizer.step()
'''
## basic train
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [46]:
##packed padded eval
'''
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
      text, text_lengths = batch.text
      text = text.to(DEVICE)
      label = batch.label.to(DEVICE)
      if(len(torch.nonzero(text_lengths))==64) : ##there are exception that length = 0 ;
        logit = model(text, text_lengths).squeeze(1)
        loss = F.cross_entropy(logit, label, reduction = 'sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(label.size()).data == label.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy
#pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
#print(pad_idx)
'''
## basic code
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy
model = RNN(1, 256, vocab_size, 128, n_classes, 0.2, True).to(DEVICE)
#model = RNN(2, 256, vocab_size, 200, 2,  pad_idx, 0.2, False)
#model = GRU(1, 256, vocab_size, 200, n_classes, 0.2).to(DEVICE)
#n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9,0.999))

In [47]:
best_val_loss = None
model = model.to(DEVICE)

for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[epoch: %d] val loss:%5.2f | val accu:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[epoch: 1] val loss: 0.37 | val accu:83.53
[epoch: 2] val loss: 0.35 | val accu:84.59
[epoch: 3] val loss: 0.36 | val accu:85.27
[epoch: 4] val loss: 0.41 | val accu:84.90
[epoch: 5] val loss: 0.52 | val accu:84.73
[epoch: 6] val loss: 0.62 | val accu:84.46
[epoch: 7] val loss: 0.69 | val accu:84.26
[epoch: 8] val loss: 0.79 | val accu:84.25
[epoch: 9] val loss: 0.79 | val accu:84.00
[epoch: 10] val loss: 0.86 | val accu:84.06
[epoch: 11] val loss: 0.85 | val accu:84.03
[epoch: 12] val loss: 0.92 | val accu:84.06
[epoch: 13] val loss: 0.98 | val accu:84.18
[epoch: 14] val loss: 0.93 | val accu:84.20
[epoch: 15] val loss: 0.94 | val accu:84.24
[epoch: 16] val loss: 0.97 | val accu:84.26
[epoch: 17] val loss: 0.99 | val accu:84.48
[epoch: 18] val loss: 0.99 | val accu:84.18
[epoch: 19] val loss: 1.02 | val accu:84.28
[epoch: 20] val loss: 1.01 | val accu:83.81


In [48]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('test loss: %5.2f | test accu: %5.2f' % (test_loss, test_acc))

test loss:  0.35 | test accu: 84.53


In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[epoch: %d] val loss:%5.2f | val accu:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('test loss: %5.2f | test accu: %5.2f' % (test_loss, test_acc))

test loss:  0.49 | test accu: 78.17
